## quora insincere questions classification
by snehasish pradhan

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"snehasish","key":"b0e3a7fe34303c3b727e65a625ecdb83"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c quora-insincere-questions-classification

 61% 3.63G/5.96G [01:44<01:51, 22.4MB/s]
User cancelled operation


In [0]:
ls

embeddings.zip  sample_data/               test.csv.zip
kaggle.json     sample_submission.csv.zip  train.csv.zip


## Libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

In [0]:
train_data = pd.read_csv("train.csv.zip")
test_data = pd.read_csv("test.csv.zip")

In [0]:
train_data.shape,test_data.shape

((1306122, 3), (56370, 2))

In [0]:
train_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [0]:
train_data.question_text.fillna('NotAvailabe',inplace=True)
test_data.question_text.fillna('NotAvailabe',inplace=True)

## Removing Symbols

In [0]:
cols_to_check = ['question_text']

In [0]:
train_data[cols_to_check] = train_data[cols_to_check].replace({'#':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({';':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({',':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({':':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'@':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'!':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'&':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'\?':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'\$':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'-':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'\]':''}, regex=True)
train_data[cols_to_check] = train_data[cols_to_check].replace({'\[':''}, regex=True)

In [0]:
test_data[cols_to_check] = test_data[cols_to_check].replace({'#':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({';':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({',':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({':':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'@':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'!':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'&':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'\?':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'\$':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'-':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'\]':''}, regex=True)
test_data[cols_to_check] = test_data[cols_to_check].replace({'\[':''}, regex=True)

## converting to lower case

In [0]:
train_data[cols_to_check] = train_data[cols_to_check].apply(lambda x: x.astype(str).str.lower())
test_data[cols_to_check] = test_data[cols_to_check].apply(lambda x: x.astype(str).str.lower())

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stop_words = set(stopwords.words('english'))

## Remove stopwords

In [0]:
train_data['question_text'] = train_data['question_text'].apply(lambda x : ' '.join(term for term in x.split() if term not in stop_words))
test_data['question_text'] = test_data['question_text'].apply(lambda x : ' '.join(term for term in x.split() if term not in stop_words))

In [0]:
train_data.head()

,qid,question_text,target
0,00002165364db923c7e6,quebec nationalists see province nation 1960s,0
1,000032939017120e6e44,adopted dog would encourage people adopt shop,0
2,0000412ca6e4628ce2cf,velocity affect time velocity affect space geo...,0
3,000042bf85aa498cd78e,otto von guericke used magdeburg hemispheres,0
4,0000455dfa3e01eae3af,convert montra helicon mountain bike changing ...,0


In [0]:
frames =[train_data['question_text'],test_data['question_text']]

In [0]:
data_text = pd.concat(frames)

In [0]:
num_words = 70000

## Tokenization and indexing

In [0]:
tokenizer = Tokenizer(num_words = num_words)

In [0]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 31.8 s, sys: 52.4 ms, total: 31.9 s
Wall time: 31.9 s


In [0]:
train_data['question_text'] = tokenizer.texts_to_sequences(train_data['question_text'])
test_data['question_text'] = tokenizer.texts_to_sequences(test_data['question_text'])

In [0]:
train_data.head()

,qid,question_text,target
0,00002165364db923c7e6,"[6602, 7174, 72, 5879, 1095, 8385]",0
1,000032939017120e6e44,"[3661, 380, 3, 3558, 4, 2987, 1704]",0
2,0000412ca6e4628ce2cf,"[1886, 265, 13, 1886, 265, 354, 5411]",0
3,000042bf85aa498cd78e,"[13124, 8427, 62842, 43, 41378, 29092]",0
4,0000455dfa3e01eae3af,"[1003, 44669, 2975, 1405, 1365, 9206]",0


In [0]:
num_tokens = [len(tokens) for tokens in train_data['question_text']]
num_tokens = np.array(num_tokens)
max_tokens = np.max(num_tokens)
max_tokens

176

In [0]:
train_data.shape,test_data.shape

((1306122, 3), (56370, 2))

In [0]:
pad = 'pre'

## Padding

In [0]:
train_pad = pad_sequences(train_data['question_text'], maxlen = max_tokens, padding = pad, truncating = pad)


In [0]:
test_pad = pad_sequences(test_data['question_text'], maxlen = max_tokens, padding = pad, truncating = pad)

In [0]:
train_pad.shape,test_pad.shape

((1306122, 176), (56370, 176))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
token_train, token_val,token_train_labl, token_val_labl = train_test_split(train_pad,train_data['target'], test_size=0.2,random_state=1)

In [0]:
token_train.shape, token_val.shape,token_train_labl.shape, token_val_labl.shape

((1044897, 176), (261225, 176), (1044897,), (261225,))

## LSTM model with embidding layer

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, SpatialDropout1D

In [0]:
modell=Sequential()
modell.add(Embedding(num_words,32))
modell.add(LSTM(32))
modell.add(Dense(1, activation='sigmoid'))

In [0]:
modell.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])

In [0]:
modell.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 32)          2240000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 2,248,353
Trainable params: 2,248,353
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = modell.fit(token_train, token_train_labl, epochs=1, validation_data=(token_val, token_val_labl), shuffle=False)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1044897 samples, validate on 261225 samples
Epoch 1/1
1044897/1044897 [==============================] - 5950s 6ms/step - loss: 0.1516 - acc: 0.9475 - val_loss: 0.1448 - val_acc: 0.9488


## Prediction

In [0]:
q = modell.predict(test_pad)

In [0]:
q[q>0.5]=1
q[q<0.5]=0

In [0]:
q

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [0]:
sumbit = pd.DataFrame(test_data['qid'])

In [0]:
sumbit['prediction'] = q

In [0]:
sumbit.head(20)

,qid,prediction
0,00014894849d00ba98a9,0.0
1,000156468431f09b3cae,0.0
2,000227734433360e1aae,0.0
3,0005e06fbe3045bd2a92,0.0
4,00068a0f7f41f50fc399,0.0
5,000a2d30e3ffd70c070d,0.0
6,000b67672ec9622ff761,0.0
7,000b7fb1146d712c1105,0.0
8,000d665a8ddc426a1907,0.0
9,000df6fd2229447b2969,0.0


In [0]:
sumbit.to_csv('predict.csv',index=False)

In [0]:
files.download('predict.csv')